In [57]:
import os
import shutil

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd

In [106]:
# Process Pokémon and one-hot encode the types

pokemon = pd.read_csv('../data/external/stats/pokemon.csv')
pokemon = pokemon[["#", "Name", "Type 1", "Type 2"]]

types = set(pokemon["Type 1"])
for t in types:
    pokemon["is" + str(t)] = 0

for i, p in pokemon.iterrows():
    pokemon.loc[i, "is" + p["Type 1"]] = 1

    if not pd.isna(p["Type 2"]):
        pokemon.loc[i, "is" + p["Type 2"]] = 1

# Save output
pokemon.to_csv('../data/processed/pokemon.csv')



In [107]:
# Categorise Pokémon into folders per type

data_directory_images = "../data/external/images-gen-1-6"
output_directory = "../data/processed/pokemon"

pokemon = pd.read_csv('../data/processed/pokemon.csv')

# Remove processed folder and create empty new one
try:
    shutil.rmtree(output_directory)
    os.mkdir(output_directory)
except:
    os.mkdir(output_directory)

# Create class folders for each Pokemon type
types = set(stats["Type 1"])
for t in types:
    os.mkdir(output_directory + "/" + t)

# Copy images to folders per type
# If a Pokemon has two types, it'll end up in two folders
for pokemon in os.listdir(data_directory_images):
    pokemon_id = pokemon.split('.')[0]

    # Remove Pokemon with subtypes
    if not pokemon_id.isnumeric():
        continue

    type1 = stats.loc[stats["#"] == int(pokemon_id), 'Type 1'].values[0]
    type2 = stats.loc[stats["#"] == int(pokemon_id), 'Type 2'].values[0]

    # Copy to type folder
    src = data_directory_images + "/" + pokemon
    dst = output_directory + "/" + type1 + "/" + pokemon_id + ".png"

    shutil.copyfile(src, dst)

    # If Pokemon has second type, copy there as well
    if not pd.isna(type2):
        dst = dst2 = output_directory + "/" + type2 + "/" + pokemon_id + ".png"
        shutil.copyfile(src, dst)